In [ ]:
!pip install image-classifiers==1.0.0b1
!pip install keras_applications --no-deps
!pip install tensorflow==2.10

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.regularizers import l2
print(tf.__version__)

2.10.0


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Load Adience dataset (with newly encoded age)

In [4]:
fold0_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold0_new')
fold1_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold1_new')
fold2_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold2_new')
fold3_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold3_new')
fold4_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold4_new')
train_new = fold0_new.concatenate(fold1_new)
train_new = train_new.concatenate(fold2_new)
val_new = fold3_new
test_new = fold4_new
val_new_batch = val_new.batch(1)
test_new_batch = test_new.batch(1)

### Load base model pretrained on CELEBA

In [ ]:
base_model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

- Last 40 layers are output layers, we want to freeze all layers except the last 10 layers (not including 40 output layers). <br> Hence, we make all layers except the last 50 layers non-trainable

In [ ]:
for layer in base_model.layers[:-50]:
  layer.trainable = False

### Create model for Adience based on pretrained model

In [ ]:
def create_model(base_model):
  ''' 
  Model pretrained on CELEBA has 40 outputs, each corrsponds to presence of a facial feature. 
  We continue from its output, reshape it to a vector of (40, 1) representing features extracted from the image.
  This 'feature vector' will be used to train our age_gender model.
  After a Dropout to prevent overfitting, we create 9 outputs:
  1 output for gender,
  8 outputs for age, each is a binary classifier of 'is age of this image older than {age_class}'. 
  For example, people in (8, 12) are also older than (4, 6), (0, 2) but not older than (15, 20) etc.

  Args:
        base_model: ResNet34 model pretrained on CELEBA
        
  Returns:
        new model used to train age_gender
  '''

  img = tf.keras.Input(shape=(224,224,3))

  # add data augmentation
  img = tf.keras.layers.RandomFlip(mode='horizontal_and_vertical')(img)
  img = tf.keras.layers.RandomRotation(0.1)(img)
  img = tf.keras.layers.RandomContrast(0.1)(img)

  # pass the image through base model trained on CELEBA
  base_outputs = base_model(img)

  '''
  base_outputs = [[[0.3], [0.2], ..., [last_in_batch]], # first element is outputs of first feature, in shape of (batch_size, 1) 
                  [[0.5], [0.8], ..., [last_in_batch]], # second element is outputs of second feature, in shape of (batch_size, 1) 
                  ...]
  we reshape each element in base_outputs to have shape (batch_size,)
  base_outputs become: [[0.3, 0.2, ..., last_in_batch], 
                        [0.5, 0.8, ..., last_in_batch], 
                        ...]
  now each column is a feature vector of length 40. Hence, we transpose it so that each feature vector takes a row
  '''
  base_outputs = tf.convert_to_tensor([tf.reshape(i, [-1]) for i in base_outputs])
  base_outputs = tf.transpose(base_outputs)

  # add Dropout to prevent overfitting
  base_outputs = tf.keras.layers.Dropout(0.6)(base_outputs)
  
  # 1 output for gender, 8 outputs for age, each with regularization to prevent overfitting
  gender_branch = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='gender_output')(base_outputs)
  age1 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group1')(base_outputs)
  age2 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group2')(base_outputs)
  age3 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group3')(base_outputs)
  age4 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group4')(base_outputs)
  age5 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group5')(base_outputs)
  age6 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group6')(base_outputs)
  age7 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group7')(base_outputs)
  age8 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group8')(base_outputs)
  
  model = tf.keras.Model(inputs = img,
                         outputs = [gender_branch, age1, age2, age3, age4, age5, age6, age7, age8])
  
  return model

In [ ]:
model = create_model(base_model)

### Evaluate age accuracy:
- The model has 8 outputs for age, each being whether 'the age of this image is older than {age_class}'. We combine label/prediction to give each image an age vector eg. `[1, 1, 0, 0, 0, 0, 0, 0]` if it belongs to the second age_group
- To do so, for predictions
```python
pred = [[[0.8], [0.3], ..., [last_in_batch]], # all outputs for gender in batch
          [[0.3], [0.2], ..., [last_in_batch]], # all outputs for age_group1 in batch
          [[0.6], [0.8], ..., [last_in_batch]], # all outputs for age_group2 in batch
          ...]
# We need all predictions from index 1 onwards (first one is prediction for gender), 
# then reshape each age element in predictions to have shape (batch_size,)
pred become: [[0.3, 0.2, ..., last_in_batch], # all outputs for age_group1
                [0.6, 0.8, ..., last_in_batch], # all outputs for age_group2
                ...]
# None each column is age feature for one image, we transpose it to make it take a row each
pred become: [[0.3, 0.2, ..., age_group8_output], 
                [0.6, 0.8, ..., age_group8_output], 
                ...
                [last_in_batch...................]]
# We then turn it into 0/1 binary values with threshold 0.5
pred become: [[0, 0, ..., age_group8_output], 
                [1, 1, ..., age_group8_output], 
                ...
                [last_in_batch...............]]
=> age_vec == [0, 0, ...] # length = 8
```
- for labels
```python
# We iterative over the whole dataset one example at a time. each example’s label_dict:
label_dict = {'age_group1':[[1]], 'age_group2': [[0]], ...}
# Hence, we take each value of 'age_groupi' and turn it to an age vector
=> age_vec == [1, 0, ...]
```
- accuracy calculation <br>
We only treat exact same age vector as a correct prediction, and we use sklearn accuracy score to compute this


In [11]:
from sklearn.metrics import accuracy_score

def decode_age_pred(pred):
  # tranpose to a 2D array with each row being an age_vector of an image
  # procedures described above
  age_logits = np.array([i.ravel() for i in pred[1:]]).transpose() 

  # encode sigmoid output to 0/1
  age_pred = np.where(age_logits > 0.5, 1, 0) 
  return age_pred

def decode_age_truth(ds):
  age_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'age_group_i' to a list, which is the age_vector
    age = [label_dict['age_group'+str(a)][0][0] for a in range(1,9)] 
    # append all age_vector to a list
    age_true.append(age) 
  return np.array(age_true)

def age_acc(model, val_ds):
  truths = decode_age_truth(val_ds)
  preds = model.predict(val_ds, verbose = 0)
  preds = decode_age_pred(preds)
  return accuracy_score(truths, preds)

# add above age accuracy score to log via callback
class AgeAccCallback(Callback):
  def __init__(self, val_ds, logs = {}):
    self.val_ds = val_ds
  def on_epoch_end(self, epoch, logs={}):
    logs['overall_age_accuracy'] = age_acc(self.model, self.val_ds)

### Train model
- We noticed that since age has multiple outputs, if 'gender_loss_weight' is set to 1, the model cannot learn to predict gender (accuracy around 0.5) since too much loss are generated by age and the model tend to focus on that
- Hence, we decided to set 'gender_loss_weight' higher at the first few epochs, 'gender_accuracy' will thus reach to an optimal level very soon After that, we set 'gender_loss_weight' to a small value to let the model focus more on optimizing age classification
- Since tensorflow does not provide functions to change `loss_weights` while training, we try to mimic this process using a for loop:
  - fit the model with only 1 epoch
  - compile again at the beginning of each iteration to reset `loss_weights`
- Model checkpoint cannot be used since we only fit 1 epoch each time. Hence, we manully monitor 'mean_val_age_gender_accuracy' and save the model when it increases

In [ ]:
old_mean_acc = -1
file_path = "/content/drive/MyDrive/save_model/celeba_modified_age_gen_base.h5"

for epoch_idx in range(1, 51):
  # for each iteration, set new config and compile

  # train epoch 11~50 with this config
  if epoch_idx > 10:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0003),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 0.9, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train epoch 1~10 with this config
  else:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0007),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 4, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )
  
  # train newly compiled model 1 epoch
  print('Epoch', str(epoch_idx)+'/50')
  hist = model.fit(train_new.shuffle(1024).batch(64),
                   epochs=1,
                   validation_data = val_new.shuffle(1024).batch(64),
                   callbacks = [EarlyStopping(monitor = 'val_loss', patience=5),
                                AgeAccCallback(val_new_batch),])
  
  # check whether metrics improved and save model
  new_mean_acc = (hist.history['val_gender_output_accuracy'][0] + hist.history['overall_age_accuracy'][0])/2
  if new_mean_acc > old_mean_acc:
    print('Validation mean accuracy for gender and age improved to ' + str(new_mean_acc) + ', saving model...')
    model.save(file_path)
    old_mean_acc = new_mean_acc

Epoch 1/50
118/118 [==============================] - 141s 988ms/step - loss: 10.4119 - gender_output_loss: 0.7020 - age_group1_loss: 0.4086 - age_group2_loss: 0.4851 - age_group3_loss: 0.6668 - age_group4_loss: 0.6842 - age_group5_loss: 0.7115 - age_group6_loss: 0.6130 - age_group7_loss: 0.6362 - age_group8_loss: 0.4029 - gender_output_accuracy: 0.5587 - age_group1_accuracy: 0.8677 - age_group2_accuracy: 0.8074 - age_group3_accuracy: 0.5960 - age_group4_accuracy: 0.5587 - age_group5_accuracy: 0.5307 - age_group6_accuracy: 0.6749 - age_group7_accuracy: 0.6373 - age_group8_accuracy: 0.8692 - val_loss: 8.6523 - val_gender_output_loss: 0.6543 - val_age_group1_loss: 0.2119 - val_age_group2_loss: 0.2847 - val_age_group3_loss: 0.5016 - val_age_group4_loss: 0.6654 - val_age_group5_loss: 0.6660 - val_age_group6_loss: 0.5585 - val_age_group7_loss: 0.3737 - val_age_group8_loss: 0.2375 - val_gender_output_accuracy: 0.6141 - val_age_group1_accuracy: 1.0000 - val_age_group2_accuracy: 0.9771 - val_a

#### We also tried not freezing any layers in CELEBA model (make all layers trainable) and this is the result:

In [ ]:
base_model2 = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')
model2 = create_model(base_model2)

In [ ]:
old_mean_acc = -1
file_path = "/content/drive/MyDrive/data/saved_model/full_celeba_modified_age_gen_base.h5"

for epoch_idx in range(1, 51):
  # for each iteration, set new config and compile

  # train epoch 11~50 with this config
  if epoch_idx > 10:
    model2.compile(
        optimizer=tf.keras.optimizers.Adam(0.0003),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 0.9, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train epoch 1~10 with this config
  else:
    model2.compile(
        optimizer=tf.keras.optimizers.Adam(0.0007),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 4, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train newly compiled model 1 epoch
  print('Epoch', str(epoch_idx)+'/50')
  hist = model2.fit(train_new.shuffle(1024).batch(64),
                    epochs=1,
                    validation_data = val_new.shuffle(1024).batch(64),
                    callbacks = [EarlyStopping(monitor = 'val_loss', patience=5),
                                 AgeAccCallback(val_new_batch),])
  
  # check whether metrics improved and save model
  new_mean_acc = (hist.history['val_gender_output_accuracy'][0] + hist.history['overall_age_accuracy'][0])/2
  if new_mean_acc > old_mean_acc:
    print('Validation mean accuracy for gender and age improved to ' + str(new_mean_acc) + ', saving model...')
    model2.save(file_path)
    old_mean_acc = new_mean_acc

Epoch 1/50
118/118 [==============================] - 82s 594ms/step - loss: 9.6650 - gender_output_loss: 0.5645 - age_group1_loss: 0.4137 - age_group2_loss: 0.5485 - age_group3_loss: 0.6395 - age_group4_loss: 0.6458 - age_group5_loss: 0.6799 - age_group6_loss: 0.6453 - age_group7_loss: 0.4950 - age_group8_loss: 0.4666 - gender_output_accuracy: 0.7069 - age_group1_accuracy: 0.8436 - age_group2_accuracy: 0.7118 - age_group3_accuracy: 0.6287 - age_group4_accuracy: 0.6227 - age_group5_accuracy: 0.5858 - age_group6_accuracy: 0.6276 - age_group7_accuracy: 0.8297 - age_group8_accuracy: 0.8378 - val_loss: 7.6211 - val_gender_output_loss: 0.4539 - val_age_group1_loss: 0.2315 - val_age_group2_loss: 0.3607 - val_age_group3_loss: 0.4200 - val_age_group4_loss: 0.5108 - val_age_group5_loss: 0.6133 - val_age_group6_loss: 0.5665 - val_age_group7_loss: 0.3759 - val_age_group8_loss: 0.3036 - val_gender_output_accuracy: 0.8128 - val_age_group1_accuracy: 1.0000 - val_age_group2_accuracy: 0.9771 - val_age

### Graphs for loss and accuracy against epochs

### Save and load trained model

In [5]:
model = tf.keras.models.load_model('/content/drive/MyDrive/data/saved_model/celeba_modified_age_gen_base.h5')
model_full = tf.keras.models.load_model('/content/drive/MyDrive/data/saved_model/full_celeba_modified_age_gen_base.h5')

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_1 (Functional)           [(None, 1),          21322993    ['input_4[0][0]']                
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                 

### Evaluate trained model on test data

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def gender_metrics(preds, ds):
  gender_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'gender_output' to a list
    gender = label_dict['gender_output'][0] 
    # append all gender to a list
    gender_true.append(gender) 
  gender_preds = np.where(preds[0].ravel()>0.5, 1, 0)
  return {'accuracy': accuracy_score(gender_true, gender_preds), 
          'f1': f1_score(gender_true, gender_preds), 
          'precision': precision_score(gender_true, gender_preds), 
          'recall': recall_score(gender_true, gender_preds)}

In [8]:
preds = model.predict(test_new_batch)

2244/2244 [==============================] - 44s 15ms/step


In [9]:
gender_metrics(preds, test_new_batch)

{'accuracy': 0.7415329768270945,
 'f1': 0.764801297648013,
 'precision': 0.7431048069345941,
 'recall': 0.7878028404344194}

In [12]:
age_acc(model, test_new_batch)

0.38235294117647056

In [13]:
preds = model.predict(val_new_batch)

2361/2361 [==============================] - 26s 11ms/step


In [14]:
gender_metrics(preds, val_new_batch)

{'accuracy': 0.7615417196103346,
 'f1': 0.7889013873265842,
 'precision': 0.7466288147622427,
 'recall': 0.8362480127186009}

In [16]:
age_acc(model, val_new_batch)

0.39940703091910207

#### Test on model with all layers trainable

In [17]:
preds2 = model_full.predict(test_new_batch)

2244/2244 [==============================] - 24s 10ms/step


In [18]:
gender_metrics(preds2, test_new_batch)

{'accuracy': 0.8083778966131907,
 'f1': 0.8250610252237591,
 'precision': 0.8041237113402062,
 'recall': 0.8471177944862155}

In [19]:
age_acc(model_full, test_new_batch)

0.42112299465240643

In [20]:
preds2 = model_full.predict(val_new_batch)

2361/2361 [==============================] - 24s 10ms/step


In [21]:
gender_metrics(preds2, val_new_batch)

{'accuracy': 0.8610758153324862,
 'f1': 0.8756633813495073,
 'precision': 0.8369565217391305,
 'recall': 0.9181240063593005}

In [22]:
age_acc(model_full, val_new_batch)

0.4773401101228293